In [1]:
import numpy as np
from abc import ABC, abstractmethod

In [2]:
class Predictor(ABC):
    def __init__(self, model_filename):
        self.model_filename = model_filename
        self.model = None
        self.model = self.loadModel()

    @abstractmethod
    def loadModel(self):
        pass

    @abstractmethod
    def predict(self, inputs, n_to_predict):
        pass

In [3]:
class XGBoostPredictor(Predictor):
    def loadModel(self):
        
        import xgboost as xgb

        model = xgb.Booster()
        model.load_model(self.model_filename)
        return model
    
    def predict(self, inputs, n_to_predict):

        import xgboost as xgb
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        inputs.drop("timestamp",axis=1,inplace=True)
        scaler.fit_transform(inputs.values)
        
        inputs=inputs[len(inputs)-n_to_predict:].values
        inputs=inputs.reshape(-1,1)
        inputs=scaler.transform(inputs)

        d_test = xgb.DMatrix(inputs)
        closing_price=self.model.predict(d_test)
        closing_price=closing_price.reshape(-1, 1)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price

In [7]:
from tensorflow.keras.models import load_model

class RNNPredictor(Predictor):
    def loadModel(self):
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        import pandas as pd
        
        df = inputs
        # data=df.sort_index(ascending=True,axis=0)
        # new_dataset=pd.DataFrame(index=range(0,len(df)),columns=['timestamp','close'])

        # for i in range(0,len(data)):
        #     new_dataset["timestamp"][i]=data['timestamp'][i]
        #     new_dataset["close"][i]=data["close"][i]
        # new_dataset.index=new_dataset.timestamp
        # new_dataset.drop("timestamp",axis=1,inplace=True)
        
        new_dataset = df.copy()
        new_dataset.set_index("timestamp", inplace=True)
        
        final_dataset=new_dataset.values
        scaler=MinMaxScaler(feature_range=(0,1))
        scaled_data=scaler.fit_transform(final_dataset)
        final_dataset=new_dataset.values 
        valid_data=final_dataset[-100:,:]
        inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
        inputs_data=inputs_data.reshape(-1,1)
        inputs_data=scaler.transform(inputs_data)

        X_test=[]
        for i in range(60,inputs_data.shape[0]):
            X_test.append(inputs_data[i-60:i,0])
        X_test=np.array(X_test)

        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price
    
    # def predict(self, inputs, n_to_predict):
    #     from sklearn.preprocessing import MinMaxScaler
    #     import pandas as pd

    #     # Preprocess data
    #     df = inputs.sort_index(ascending=True, axis=0)
    #     new_dataset = df[["timestamp", "close"]].copy()
    #     new_dataset.set_index("timestamp", inplace=True)
        
    #     # Fit scaler on training data and scale input data
    #     scaler = MinMaxScaler(feature_range=(0, 1))
    #     scaled_data = scaler.fit_transform(new_dataset)

    #     # Prepare input sequences
    #     inputs_data = scaled_data[-len(new_dataset) - 60:]
    #     X_test = [inputs_data[i - 60:i, 0] for i in range(60, inputs_data.shape[0])]
    #     X_test = np.array(X_test).reshape(-1, 60, 1)

    #     # Make predictions
    #     closing_price = self.model.predict(X_test)
    #     closing_price = scaler.inverse_transform(closing_price)

    #     return closing_price

In [5]:
class LSTMPredictor(Predictor):
    def loadModel(self):    
        return load_model(self.model_filename)
    
    def predict(self, inputs, n_to_predict):
        
        from sklearn.preprocessing import MinMaxScaler
        scaler=MinMaxScaler(feature_range=(0,1))
        
        inputs.drop("timestamp",axis=1,inplace=True)
        scaler.fit_transform(inputs.values)
        
        inputs=inputs[len(inputs)-n_to_predict:].values
        inputs=inputs.reshape(-1,1)
        inputs=scaler.transform(inputs)

        X_test=np.array(inputs)

        X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
        closing_price=self.model.predict(X_test)
        closing_price=scaler.inverse_transform(closing_price)
        
        return closing_price